In [1]:
# Import the libraries
import ee 
import geemap
import os
import geopandas as gpd
import osmnx as ox


In [3]:
# Trigger the authentication flow
ee.Authenticate()


Successfully saved authorization token.


In [2]:
# Initialize the library
ee.Initialize()

In [3]:
# initializes the Google Earth Engine API
geemap.ee_initialize()

# Nighttime Lights

In [4]:
# Load the boundary of gaza
adm1 = ee.FeatureCollection("FAO/GAUL/2015/level1")
gaza = adm1.filter(ee.Filter.eq("ADM0_NAME", "Gaza Strip"))
roi = gaza.geometry()

In [5]:
# Visualize the boundary
Map1= geemap.Map(center=(31.41792, 34.37081), zoom=10)
Map1.addLayer(roi)
Map1

Map(center=[31.41792, 34.37081], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [6]:
# Load the monthly nighttime data (VIIRS)

viirs = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate("2023-07-01", "2024-06-06").select("avg_rad")
print("The numbers of nighttime light is:", viirs.size().getInfo())

The numbers of nighttime light is: 7


In [7]:
# Filter the nighttime lights between October 2022 and April 2023
nighttime_collection_pre = viirs.filterDate("2023-07-01", "2023-09-30")
nighttime_image_pre = nighttime_collection_pre.max().clip(roi)


# Filter the nighttime lights between October 2023 and April 2024
nighttime_collection_post = viirs.filterDate("2023-10-07", "2024-06-06")
nighttime_image_post = nighttime_collection_post.max().clip(roi)

print("The numbers of nighttime light before:", nighttime_collection_pre.size().getInfo())
print("The numbers of nighttime light after:", nighttime_collection_post.size().getInfo())

The numbers of nighttime light before: 3
The numbers of nighttime light after: 3


## Sentinel-2 Images

In [8]:
# Load Sentinel-2 image collection
s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                    .filterDate("2023-07-01", "2024-06-06")
                    .filterMetadata("CLOUDY_PIXEL_PERCENTAGE", "less_than", 30)
                    .filterBounds(roi))

In [9]:
# Filter the s2 collection between July - September 2023 
s2_collection_pre = s2.filterDate("2023-07-01", "2023-09-30").select(["B.*"])


# Filter the s2 collection between October 07 2023 - June 06 2024
s2_collection_post = s2.filterDate("2023-10-07", "2024-06-06").select(["B.*"])


In [10]:
# Create mosaics for the pre and post collections
s2_mosaic_pre = s2_collection_pre.mosaic().clip(roi)

s2_mosaic_post = s2_collection_post.mosaic().clip(roi)

In [11]:
# compute  Normalized Difference Built-up Index (NDBI)
ndbi_pre = s2_mosaic_pre.normalizedDifference(["B11", "B8"]).rename("NDBI")
ndbi_post = s2_mosaic_post.normalizedDifference(["B11", "B8"]).rename("NDBI")

ndbi_pre = ndbi_pre.select("NDBI")
ndbi_post = ndbi_post.select("NDBI")

## Building Footprints

In [12]:
# Convert the gaza boundary to a GeoDataFrame
gaza_gdf= geemap.ee_to_gdf(gaza)

gaza_gdf.head()

,geometry,ADM0_CODE,ADM0_NAME,ADM1_CODE,ADM1_NAME,DISP_AREA,EXP1_YEAR,STATUS,STR1_YEAR,Shape_Area,Shape_Leng
0,"POLYGON ((34.31211 31.40379, 34.31299 31.40317...",91,Gaza Strip,1291,Deir al Balah,NO,3000,Occupied Palestinan Territory,1000,0.005387,0.339650
1,"POLYGON ((34.37770 31.46675, 34.37866 31.46578...",91,Gaza Strip,1292,Gaza,NO,3000,Occupied Palestinan Territory,1000,0.007043,0.430250
2,"POLYGON ((34.45635 31.55177, 34.45666 31.54884...",91,Gaza Strip,1293,Jabalya,NO,3000,Occupied Palestinan Territory,1000,0.005757,0.327872
3,"POLYGON ((34.24102 31.33973, 34.24148 31.33883...",91,Gaza Strip,1294,Khan Yunis,NO,3000,Occupied Palestinan Territory,1000,0.010131,0.449412
4,"POLYGON ((34.21980 31.32169, 34.23301 31.30140...",91,Gaza Strip,1295,Rafah,NO,3000,Occupied Palestinan Territory,1000,0.006056,0.344584


In [13]:
# Convert gaza GeoDataFrame to a Polygon
if gaza_gdf.geometry.count() > 1:
        gaza_poly = gaza_gdf.unary_union
else:
        gaza_poly = gaza_gdf.geometry.iloc[0]

In [14]:
# Load building footprints from OSM
tag = {"building": True}

bdg = ox.features_from_polygon(gaza_poly, tags=tag)

In [15]:
# Inspect the building footprints GeoDataFrame
bdg.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 280005 entries, ('node', 503993244) to ('relation', 17655836)
Columns: 135 entries, building to website:menu
dtypes: geometry(1), object(134)
memory usage: 299.9+ MB


In [16]:
# Check the columns in  building footprints GeoDataFrame
bdg.columns

Index(['building', 'name', 'name:ar', 'geometry', 'man_made', 'name:pt',
       'fixme', 'shop', 'name:en', 'source',
       ...
       'building:flats', 'tower:construction', 'tower:type', 'place',
       'substance', 'greenhouse', 'ways', 'type', 'emergency', 'website:menu'],
      dtype='object', length=135)

In [17]:
bdg.head(2)

building                       name  \
element_type osmid                                           
node         503993244   public  Appex Pharma for Medicine   
             503993843      yes                        NaN   

                                    name:ar                   geometry  \
element_type osmid                                                       
node         503993244  أبيكس فارما للأدوية  POINT (34.46417 31.54045)   
             503993843                  NaN  POINT (34.46877 31.55191)   

                       man_made name:pt fixme shop name:en source  ...  \
element_type osmid                                                 ...   
node         503993244      NaN     NaN   NaN  NaN     NaN    NaN  ...   
             503993843      NaN     NaN   NaN  NaN     NaN    NaN  ...   

                       building:flats tower:construction tower:type place  \
element_type osmid                                                          
node         503993244            NaN                NaN        NaN   NaN   
             503993843            NaN                NaN        NaN   NaN   

                       substance greenhouse ways type emergency website:menu  
element_type osmid                                                            
node         503993244       NaN        NaN  NaN  NaN       NaN          NaN  
             503993843       NaN        NaN  NaN  NaN       NaN          NaN  

[2 rows x 135 columns]

In [18]:
# Reset the index
bdg = bdg.reset_index()
bdg.head(2)

,element_type,osmid,building,name,name:ar,geometry,man_made,name:pt,fixme,shop,...,building:flats,tower:construction,tower:type,place,substance,greenhouse,ways,type,emergency,website:menu
0,node,503993244,public,Appex Pharma for Medicine,أبيكس فارما للأدوية,POINT (34.46417 31.54045),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,503993843,yes,NaN,NaN,POINT (34.46877 31.55191),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Convert the building footprints to GEE feature collection
bdg_filter = bdg[["osmid", "building","name", "name:ar", "geometry"]]

bdg_ee = geemap.gdf_to_ee(bdg_filter)

print(type(bdg_ee))

<class 'ee.featurecollection.FeatureCollection'>


## Visualization

In [20]:
# Visualize the building footprints alone 
# THIS WILL TAKE A LOT OF TIME TO DISPLAY
#bdg.explore()

In [21]:
# Define the visualiztion parameters for the nighttime lights, sentinel-2 images, and building footprints
vis_params = [
    {"min": 0.0, "max": 20.0, "palette":["#000000", "#f7f6bb", "#ffba08"]},
    {"min": 0.0, "max" : 20.0, "palette":["#000000", "#f7f6bb", "#ffba08"]},
    {"min": 0, "max": 3000, "bands": ["B4", "B3", "B2"], "gamma": 0.95, },
    {"min": 0, "max": 3000, "bands": ["B4", "B3", "B2"], "gamma": 0.95},
]

In [22]:
# Define the labels for the maps
labels = [
    "Nighttime Lights July - September 2023",
    "Nighttime Lights October 2023 - June 2024",
    "Sentinel-2 July - September 2023",
    "Sentinel-2 October 2023 - June 2024",
]

In [23]:
# Display the nighttime lights and sentinel-2 images before and after the event
geemap.linked_maps(
    rows=2,
    cols=2,
    height="450px",
    center=[31.41792, 34.37081],
    zoom=10,
    ee_objects=[nighttime_image_pre, nighttime_image_post, s2_mosaic_pre, s2_mosaic_post],
    vis_params=vis_params,
    labels=labels,
    label_position="topright",
)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…